In [19]:
import pandas as pd
import pvlib
from pvlib.location import Location
from pvlib.pvsystem import PVSystem
from pvlib.modelchain import ModelChain
from dateutil.tz import gettz
from dateutil import tz
import pytz
import numpy as np

In [25]:
#参数设置
latitude = 22.81    # 替换为您的纬度
longitude = 109.445 # 替换为您的经度
altitude = 100.0     # 添加海拔高度（单位：米）

pdc0=26000         # 标准测试条件功率 [W]
gamma_pdc= -0.0034     # 温度系数
temp_noct=20      # NOCT 温度
eta_inv_nom=0.85  #逆变器转化率

surface_tilt=6  #光伏板倾斜角
surface_azimuth=177   #光伏板方位角
albedo=0.20  #反射率，一般默认为0.2  雪地环境为0.8

In [26]:

# 1. 设置地理位置和时区（北京时间 UTC+8）
latitude = latitude    # 替换为您的纬度
longitude = longitude # 替换为您的经度
altitude = altitude     # 添加海拔高度（单位：米）
timezone = pytz.timezone('Asia/Shanghai')  # 使用pytz时区对象

location = pvlib.location.Location(latitude, longitude, tz=timezone, altitude=altitude)

# 2. 读取气象数据（包含时间戳和GHI列）
weather = pd.read_excel(
    r"D:\GHI预测\f广西\122发电量\122GHI0701-0731.xlsx",
    parse_dates=['datetime'],
    index_col='datetime'
)

# 将时间戳本地化为北京时间
weather.index = weather.index.tz_localize(timezone)

# 简单测试：确保 GHI 列存在 且 索引带有时区信息
assert 'ghiradiation' in weather.columns, "Excel 文件中未找到 GHI 列，请检查列名"
assert weather.index.tz is not None, "索引未被正确本地化为带时区的 DateTimeIndex"
assert 'temp' in weather.columns, "Excel 文件中未找到温度列，请检查列名"

# 重命名 solarradiation 列为 ghi
weather.rename(columns={'ghiradiation': 'ghi'}, inplace=True)
weather.rename(columns={'dniradiation': 'dni'}, inplace=True)

# 3. 从 GHI 和DNI 计算 DHI
# 获取太阳位置信息
solar_position = pvlib.solarposition.get_solarposition(
    time=weather.index, 
    latitude=latitude, 
    longitude=longitude, 
    altitude=altitude, 
    pressure=None, 
    method='nrel_numpy', 
    temperature=weather['temp']  # 提供温度值
)

# 获取太阳天顶角
solar_zenith = solar_position['zenith']

# 确保没有负值或过大值
weather['ghi'] = weather['ghi'].clip(lower=0, upper=1500)
weather['dni'] = weather['dni'].clip(lower=0, upper=1500)

cos_zenith = np.cos(np.radians(solar_zenith))
cos_zenith[cos_zenith < 0] = 0  # 太阳在地平线以下时强制为 0

weather['dhi'] = weather['ghi'] - weather['dni'] * cos_zenith
weather['dhi'] = weather['dhi'].clip(lower=0, upper=weather['ghi'])  # 再次修正不合理值

# 4. 确保所有必要的气象数据列都存在
required_cols = ['ghi', 'dni', 'dhi', 'temp']
for col in required_cols:
    if col not in weather.columns:
        weather[col] = 0  # 添加缺失列并设为0

# 添加风速列（如果不存在）
# if 'windspeed' not in weather.columns:
#     weather['windspeed'] = 1.0  # 默认风速

# 5. 定义组件和逆变器参数，以及 PVSyst 温度模型参数
module_parameters = {
    'pdc0': pdc0,         # 标准测试条件功率 [W]
    'gamma_pdc': gamma_pdc, # 温度系数
    'temp_noct': temp_noct      # NOCT 温度
}
inverter_parameters = {
    'pdc0': pdc0,
    'eta_inv_nom':0.85  #逆变器转化率
}
temperature_model_parameters = {
    'a': -3.56,    # PVSyst 模型参数 a
    'b': -0.075,   # PVSyst 模型参数 b
    'deltaT': 3    # PVSyst 模型参数 ΔT
}

system = PVSystem(
    surface_tilt=surface_tilt,
    surface_azimuth=surface_azimuth,
    module_parameters=module_parameters,
    inverter_parameters=inverter_parameters,
    temperature_model_parameters=temperature_model_parameters,
    albedo=albedo
)

# 6. 创建 ModelChain，使用 PVWatts 直流模型和 PVWatts 交流模型
mc = ModelChain(
    system=system,
    location=location,
    clearsky_model='ineichen',
    transposition_model='haydavies',
    solar_position_method='nrel_numpy',
    airmass_model='kastenyoung1989',
    aoi_model='physical',
    dc_model='pvwatts',  # 使用 PVWatts 直流模型
    ac_model='pvwatts',  # 使用 PVWatts 交流模型
    spectral_model='no_loss',
    name='standard_system'
)

# 7. 运行模型
mc.run_model(weather)

# 8. 获取结果
results = pd.DataFrame({
    'Datetime': weather.index,
    'GHI': weather['ghi'],
    'DNI': weather['dni'],
    'DHI': weather['dhi'],
    'Temperature': weather['temp'],
    'DC_Power': mc.results.dc,
    'AC_Power': mc.results.ac
})

# 9. 计算每小时发电量 (Wh)
results['Hourly_Energy_Wh'] = (results['AC_Power'] * 1.0 )/1000 # 功率(W) * 时间(h) = 能量(Wh)再/1000换算成kwh的单位 1kwh=1度电

# 10. 保存结果
results.to_csv('pv_system_results3.csv', index=False)
print("模拟结果已保存到 pv_system_results3.csv")

模拟结果已保存到 pv_system_results3.csv
